In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn
import numpy as np

import dataset
import rendering
import model
import train

import mcubes
import trimesh
from model import NeRFLightning
from config import (
    CKPT_DIR,
    SCALE,
    NB_BINS
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lit_nerf = NeRFLightning.load_from_checkpoint(CKPT_DIR)
lit_nerf.eval()
N = NB_BINS
x = torch.linspace(-SCALE, SCALE, N)
y = torch.linspace(-SCALE, SCALE, N)
z = torch.linspace(-SCALE, SCALE, N)

# 3d grid values
x, y, z = torch.meshgrid((x, y, z))
xyz = torch.cat((x.reshape(-1, 1), y.reshape(-1, 1), z.reshape(-1, 1)), dim=1)

In [ ]:
chunk_size = 10000
chunks = [xyz[i:i + chunk_size] for i in range(0, xyz.shape[0], chunk_size)]
densities = []
for chunk in chunks:
  
  with torch.inference_mode():
    _, density = lit_nerf.forward(chunk.to(device), torch.zeros_like(chunk).to(device))
  densities.append(density)
density = torch.cat(densities, dim=0)
density = density.cpu().numpy().reshape(N, N, N)

In [ ]:
vertices, triangles = mcubes.marching_cubes(density, 10 * np.mean(density))
mesh = trimesh.Trimesh(vertices / N, triangles)
mesh.show()

In [ ]:
mesh.export("models/nerf_mesh.obj")

In [ ]:
trimesh.load_mesh("models/nerf_mesh.obj").show()